In [ ]:
# TODO: caching

In [18]:
# !wget https://pages.cs.wisc.edu/~harter/cs639/data/ghcnd-stations.txt

# import requests
# count = 0
# for l in open("ghcnd-stations.txt"):
#     state = l[38:40]
#     station = l[:11]
    
#     if state == "WI":
#         count += 1
#         print(count, state, station)
#         r = requests.get(f"https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_station/{station}.csv.gz")
#         r.raise_for_status()
#         with open(f"{station}.csv.gz", "wb") as f:
#             f.write(r.content)
# count

--2023-01-26 19:45:20--  https://pages.cs.wisc.edu/~harter/cs639/data/ghcnd-stations.txt
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10607756 (10M) [text/plain]
Saving to: ‘ghcnd-stations.txt’

ghcnd-stations.txt  100%[===================>]  10.12M  35.8MB/s    in 0.3s    

2023-01-26 19:45:21 (35.8 MB/s) - ‘ghcnd-stations.txt’ saved [10607756/10607756]



In [78]:
import requests
r = requests.get("https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/stations.txt")
r.raise_for_status()
stations = r.text.strip().split("\n")
stations = random.sample(stations, k=10)
workload = random.choices(stations, k=100, weights=[0.3, 0.2] + [0.5/8]*8)

In [100]:
import time, requests

cache_size = 3
cache = {}
evict_order = []
latencies = []
hits = []

def get_station(station):
    t0 = time.time()
    if station in cache:
        hits.append(1)
        evict_order.remove(station)
        evict_order.append(station)
        df = cache[station]
    else:
        hits.append(0)

        # add to cache
        df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/{station}.csv.gz",
                         names=["station", "date", "element", "value", "m", "q", "s", "obs"])
        cache[station] = df
        evict_order.append(station)

        # evict from cache (if needed)
        if len(cache) > cache_size:
            victim = evict_order.pop(0)
            cache.pop(victim)
    
    t1 = time.time()
    latencies.append((t1 - t0) * 1000)
    return df

for station in workload:
    get_station(station)

In [101]:
sum(hits) / len(hits)

0.4

In [94]:
sum(latencies) / len(latencies)

46.946349143981934

In [97]:
import numpy as np
list(np.quantile(latencies, [0.5, 0.99]))

[0.001430511474609375, 245.49561023712158]

In [102]:
pd.read_csv("https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/US1WIAD0002.csv.gz")

,US1WIAD0002,20080501,PRCP,76,Unnamed: 4,Unnamed: 5,N,1000
0,US1WIAD0002,20080502,PRCP,0,NaN,NaN,N,1000
1,US1WIAD0002,20080503,PRCP,0,NaN,NaN,N,1000
2,US1WIAD0002,20080504,PRCP,0,NaN,NaN,N,1000
3,US1WIAD0002,20080505,PRCP,0,NaN,NaN,N,1000
4,US1WIAD0002,20080506,PRCP,160,NaN,NaN,N,1000
...,...,...,...,...,...,...,...,...
206,US1WIAD0002,20080814,SNOW,0,NaN,NaN,N,900
207,US1WIAD0002,20080815,SNOW,0,NaN,NaN,N,900
208,US1WIAD0002,20080816,SNOW,0,NaN,NaN,N,900
209,US1WIAD0002,20080817,SNOW,0,NaN,NaN,N,900


1 WI US1WIAD0002
2 WI US1WIAD0005
3 WI US1WIAD0006
4 WI US1WIAD0008
5 WI US1WIAD0010
6 WI US1WIAD0015
7 WI US1WIAD0016
8 WI US1WIAD0017
9 WI US1WIAS0003
10 WI US1WIAS0004
11 WI US1WIAS0006
12 WI US1WIAS0011
13 WI US1WIAS0012
14 WI US1WIAS0017
15 WI US1WIBF0002
16 WI US1WIBF0003
17 WI US1WIBN0001
18 WI US1WIBN0006
19 WI US1WIBN0007
20 WI US1WIBN0008
21 WI US1WIBN0010
22 WI US1WIBN0012
23 WI US1WIBN0014
24 WI US1WIBN0015
25 WI US1WIBN0018
26 WI US1WIBN0020
27 WI US1WIBN0022
28 WI US1WIBN0029
29 WI US1WIBN0031
30 WI US1WIBN0032
31 WI US1WIBR0002
32 WI US1WIBR0006
33 WI US1WIBR0007
34 WI US1WIBR0008
35 WI US1WIBR0011
36 WI US1WIBR0012
37 WI US1WIBR0013
38 WI US1WIBR0015
39 WI US1WIBR0016
40 WI US1WIBR0017
41 WI US1WIBR0018
42 WI US1WIBR0019
43 WI US1WIBR0020
44 WI US1WIBR0021
45 WI US1WIBR0022
46 WI US1WIBR0023
47 WI US1WIBR0026
48 WI US1WIBR0027
49 WI US1WIBR0030
50 WI US1WIBR0031
51 WI US1WIBR0032
52 WI US1WIBR0037
53 WI US1WIBR0038
54 WI US1WIBR0041
55 WI US1WIBT0001
56 WI US1WIBT0002
5

1313

In [1]:
!wget https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations.zip

--2023-01-26 19:26:59--  https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations.zip
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10607756 (10M) [application/zip]
Saving to: ‘wi-stations.zip’

wi-stations.zip     100%[===================>]  10.12M  17.0MB/s    in 0.6s    

2023-01-26 19:27:00 (17.0 MB/s) - ‘wi-stations.zip’ saved [10607756/10607756]



In [6]:
! unzip -l wi-stations.zip

Archive:  wi-stations.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of wi-stations.zip or
        wi-stations.zip.zip, and cannot find wi-stations.zip.ZIP, period.


In [9]:
import pandas as pd

In [15]:
!tar -xfzg US1WIAD0005.tar.gz

tar: zg: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [10]:
pd.read_csv("https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/US1WIAD0005.tar.gz")

ReadError: file could not be opened successfully:
- method gz: ReadError('invalid header')
- method bz2: ReadError('not a bzip2 file')
- method xz: ReadError('not an lzma file')
- method tar: ReadError('invalid header')